In [3]:
%%capture
!pip install pytube

In [4]:
from pytube import Playlist, YouTube, Stream ,Search

In [9]:
urls = [
        "https://www.youtube.com/playlist?list=PLZDMziPRHWIrkfzKIXUOrir_gwD0cD9f8" , # test
        "https://www.youtube.com/playlist?list=PLR-r0edywujc9bj3vpNjqIDg-WbWj3vyz" , # 7
        "https://youtube.com/playlist?list=PLFsQleAWXsj_4yDeebiIADdH5FMayBiJo" ,     # 300
        "https://youtube.com/playlist?list=PL4lEESSgxM_5O81EvKCmBIm_JT5Q7JeaI" ,     # 800
        "https://youtube.com/playlist?list=PLG9huCBGQQq33BxkisPUvWx8mUNOVyg4w" ,     # 5000
        ]

In [10]:
plurl = "https://www.youtube.com/playlist?list=PLFsQleAWXsj_4yDeebiIADdH5FMayBiJo" #310
# https://www.youtube.com/watch?v=q6EoRBvdVPQ # 1
# https://www.youtube.com/watch?v=ixQkcuZhXg8 # copyright
# https://www.youtube.com/watch?v=EWF8Nfm-LLk # api error
# https://www.youtube.com/watch?v=FahK9srJq10 # not available

In [ ]:
# plurl = "https://youtube.com/playlist?list=PLZDMziPRHWIrkfzKIXUOrir_gwD0cD9f8" #test
# https://www.youtube.com/watch?v=FtdpDjV0dMY # 1
# https://www.youtube.com/watch?v=QvW61K2s0tA # copyright

In [ ]:
p = Playlist(plurl)

In [ ]:
v = p.videos[7]

In [ ]:
s = v.streams[0]

In [ ]:
# s.download()

#### dir

In [ ]:
dir(p)

In [ ]:
for att in dir(p):
    print (att, "=============>>" ,getattr(p,att))

In [ ]:
dir(v)

In [ ]:
with open("dsvv.txt" ,"w") as f:
  for att in dir(v):
      f.write(f"{att}  =============>>  {getattr(v,att)}\n")

In [ ]:
dir(s)

In [ ]:
for att in dir(s):
    print (att, "=============>>" ,getattr(s,att))

#### vars

In [ ]:
vars(p)

In [ ]:
vars(v)

In [ ]:
vars(s)

#### Parallel Scraping 

##### single playlist

In [ ]:
%%time
import re
from tqdm.notebook import tqdm
pattern = r'"videoDurationSeconds\\":\\"([0-9]+)\\",'
durs = []
for v in tqdm(p.videos,total=len(p.videos)):
  op = re.findall(pattern, v.embed_html)
  if not op == []:
    durs.append(int(op[0]))
  else:
    durs.append(int(v.length))
print(p.title)
print(len(durs))
print(sum(durs))

  0%|          | 0/306 [00:00<?, ?it/s]

important videos
306
11698
CPU times: user 1.89 s, sys: 414 ms, total: 2.31 s
Wall time: 24.6 s


##### single playlist with multipools

In [ ]:
%%time
from multiprocessing import Pool
import re
pattern = r'"videoDurationSeconds\\":\\"([0-9]+)\\",'

def vtodur(v):
  op = re.findall(pattern, v.embed_html)
  return int(op[0]) if not op == [] else int(v.length)

with Pool(10) as pp:
  durs = pp.map(vtodur,p.videos)

print(p.title)
print(len(durs))
print(sum(durs))

important videos
306
11698
CPU times: user 88.1 ms, sys: 122 ms, total: 210 ms
Wall time: 3.06 s


##### multi playlist

In [ ]:
%%time
import re
from tqdm.notebook import tqdm
pattern = r'"videoDurationSeconds\\":\\"([0-9]+)\\",'
def fuc(plurl):
  p = Playlist(plurl)
  durs = []
  for v in tqdm(p.videos,total=len(p.videos)):
    op = re.findall(pattern, v.embed_html)
    if not op == []:
      durs.append(int(op[0]))
    else:
      durs.append(int(v.length))
  print(p.title)
  print(len(durs))
  print(sum(durs))

for i in urls[3:]:
  fuc(i)
  print()

##### multi playlist with multipool and process for length > 100

In [ ]:
%%time
import re
from multiprocessing import Pool, Process
from tqdm.notebook import tqdm
pattern = r'"videoDurationSeconds\\":\\"([0-9]+)\\",'

def vtodur(v):
  op = re.findall(pattern, v.embed_html)
  return int(op[0]) if not op == [] else int(v.length)

def ptodur(p):
  with Pool(10) as pp:
    durs = pp.map(vtodur,p.videos)
  print(p.title)
  print(len(durs))
  print(sum(durs))

for purl in tqdm(urls[3:],total=len(urls)):
  p = Playlist(purl)
  if len(p) > 100 :
    p1 = Process(target=ptodur,args=(p,))
    p1.start()
    p1.join()
  else : 
    ptodur(p)
  print()

##### multi playlist with multipool and process for length > 1000

In [ ]:
%%time
import re
from multiprocessing import Pool, Process
from tqdm.notebook import tqdm
pattern = r'"videoDurationSeconds\\":\\"([0-9]+)\\",'

def vtodur(v):
  op = re.findall(pattern, v.embed_html)
  return int(op[0]) if not op == [] else int(v.length)

def ptodur(p):
  with Pool(10) as pp:
    durs = pp.map(vtodur,p.videos)
  print(p.title)
  print(len(durs))
  print(sum(durs))

for purl in tqdm(urls[3:],total=len(urls)):
  p = Playlist(purl)
  if len(p) > 1000 :
    p1 = Process(target=ptodur,args=(p,))
    p1.start()
    p1.join()
  else : 
    ptodur(p)
  print()

  0%|          | 0/5 [00:00<?, ?it/s]

THE LONGEST MIXTAPE - 1000 Songs For You
814
286535

A playlist with 5000 videos
4999
401180

CPU times: user 3.02 s, sys: 518 ms, total: 3.54 s
Wall time: 1min 22s


#### Usefulls

In [ ]:
v.initial_data

{'responseContext': {'serviceTrackingParams': [{'service': 'CSI',
    'params': [{'key': 'c', 'value': 'WEB'},
     {'key': 'cver', 'value': '2.20230201.01.00'},
     {'key': 'yt_li', 'value': '0'},
     {'key': 'GetWatchNext_rid', 'value': '0xf20db086ecc38dc9'}]},
   {'service': 'GFEEDBACK',
    'params': [{'key': 'logged_in', 'value': '0'},
     {'key': 'e',
      'value': '1714244,23804281,23848225,23858057,23918597,23946420,23966208,23983296,23986028,23998056,24004644,24007246,24034168,24036948,24077241,24080738,24120819,24135310,24140247,24161116,24162919,24164186,24166867,24169501,24181174,24186126,24187043,24187377,24197450,24209350,24211178,24219713,24228033,24240253,24241378,24255163,24255543,24255545,24262346,24263796,24267564,24268142,24281897,24282010,24283015,24283093,24288045,24288664,24290971,24291857,24294555,24390674,24391543,24396645,24401013,24404640,24406314,24407190,24414719,24415864,24415866,24416291,24419345,24426636,24428940,24429093,24429868,24430382,24433679,2

In [ ]:
v.streaming_data

{'expiresInSeconds': '21540',
 'formats': [{'itag': 18,
   'url': 'https://rr5---sn-qxo7rn7r.googlevideo.com/videoplayback?expire=1675688201&ei=qaTgY_HaHtmdsfIPlKySuAM&ip=35.230.71.223&id=o-AJt7oDvKRxs3LM_xTPONqAtgng_im6HV4NofufhT_sUK&itag=18&source=youtube&requiressl=yes&mh=QP&mm=31%2C26&mn=sn-qxo7rn7r%2Csn-5go7ynld&ms=au%2Conr&mv=u&mvi=5&pl=20&vprv=1&mime=video%2Fmp4&gir=yes&clen=10130610015&ratebypass=yes&dur=720000.383&lmt=1596607579855703&mt=1675666434&fvip=2&fexp=24007246&c=ANDROID&txp=6216222&sparams=expire%2Cei%2Cip%2Cid%2Citag%2Csource%2Crequiressl%2Cvprv%2Cmime%2Cgir%2Cclen%2Cratebypass%2Cdur%2Clmt&sig=AOq0QJ8wRgIhALPdNvnRoJgwcktT-sTgHYYKKFqEfs__KVXZAKNM3c-FAiEA7sn4gG2hCdi_HUJVJ5p3dReLBwTMbAUWNCpPrKtMXCk%3D&lsparams=mh%2Cmm%2Cmn%2Cms%2Cmv%2Cmvi%2Cpl&lsig=AG3C_xAwRgIhAKldYjSoulHn3LyrXbSRmPLV0Q7rQtrjZ406jBICM3naAiEA1KLULSH4lEcLJyt9MZgRhP_ZIUBHrPaqxgMF9rA2DSI%3D',
   'mimeType': 'video/mp4; codecs="avc1.42001E, mp4a.40.2"',
   'bitrate': 112562,
   'width': 320,
   'height': 180

In [ ]:
v.vid_info

{'responseContext': {'visitorData': 'Cgs0UkFoRTlYejYtcyipyYKfBg%3D%3D',
  'serviceTrackingParams': [{'service': 'GFEEDBACK',
    'params': [{'key': 'ipcc', 'value': '0'},
     {'key': 'is_alc_surface', 'value': 'false'},
     {'key': 'is_viewed_live', 'value': 'False'},
     {'key': 'logged_in', 'value': '0'},
     {'key': 'e',
      'value': '1714245,23888717,23918597,23946420,23966208,23983296,23998056,24004644,24007246,24034168,24036948,24077241,24080738,24117491,24120820,24123943,24135310,24140247,24161116,24162919,24164186,24169501,24181174,24187043,24187377,24197450,24209349,24211178,24219713,24240253,24241378,24255165,24255543,24255545,24262346,24263796,24265964,24267564,24268142,24272792,24281897,24283015,24283093,24285788,24288047,24288664,24290971,24294555,24390674,24391539,24391708,24397801,24404640,24405623,24406318,24407191,24412636,24414718,24415866,24418790,24421999,24426636,24428902,24429095,24430381,24433346,24436009,24437351,24439483,24440132,24441240,24441691,2444523